https://towardsdatascience.com/stock-prediction-using-recurrent-neural-networks-c03637437578

https://viblo.asia/p/recurrent-neural-networkphan-1-tong-quan-va-ung-dung-jvElaB4m5kw

https://nttuan8.com/bai-13-recurrent-neural-network/

https://lilianweng.github.io/lil-log/2017/07/08/predict-stock-prices-using-RNN-part-1.html

https://www.kaggle.com/raoulma/ny-stock-price-prediction-rnn-lstm-gru

In [1]:
import numpy as np
import pandas as pd
import math
import sklearn
from sklearn.preprocessing import MinMaxScaler
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

# #display parent directory and working directory
# print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
# print(os.getcwd()+':', os.listdir(os.getcwd()));

In [2]:
# import all stock prices 
df = pd.read_csv("../input/Apple.csv", index_col = 0)
df.info()
df

# # number of different stocks
# print('\nnumber of different stocks: ', len(list(set(df.symbol))))
# print(list(set(df.symbol))[:10])

<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 2016-06-13 to 2021-06-10
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       1258 non-null   float64
 1   Low        1258 non-null   float64
 2   Open       1258 non-null   float64
 3   Close      1258 non-null   float64
 4   Volume     1258 non-null   float64
 5   Adj Close  1258 non-null   float64
 6   Name       1258 non-null   object 
dtypes: float64(6), object(1)
memory usage: 78.6+ KB


,High,Low,Open,Close,Volume,Adj Close,Name
Date,,,,,,,
2016-06-13,24.780001,24.275000,24.672501,24.334999,152082000.0,22.733387,AAPL
2016-06-14,24.620001,24.187500,24.330000,24.365000,127727600.0,22.761415,AAPL
2016-06-15,24.602501,24.257500,24.455000,24.285000,117780800.0,22.686680,AAPL
2016-06-16,24.437500,24.017500,24.112499,24.387501,125307200.0,22.782436,AAPL
2016-06-17,24.162500,23.825001,24.155001,23.832500,244032800.0,22.263964,AAPL
...,...,...,...,...,...,...,...
2021-06-04,126.160004,123.849998,124.070000,125.889999,75169300.0,125.889999,AAPL
2021-06-07,126.320000,124.830002,126.169998,125.900002,71057600.0,125.900002,AAPL
2021-06-08,128.460007,126.209999,126.599998,126.739998,74403800.0,126.739998,AAPL


In [3]:
df.describe()
# df.columns=["open", "high","low", "close", "adjclose", "volume"]


,High,Low,Open,Close,Volume,Adj Close
count,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258.000000
mean,61.447436,60.133203,60.788158,60.815085,1.270491e+08,59.641747
std,32.877387,31.920018,32.449177,32.399088,5.748357e+07,32.866308
min,23.262501,22.875000,23.225000,23.010000,4.544800e+07,21.495594
25%,39.184999,38.516251,38.948124,38.873751,8.862690e+07,37.262375
50%,48.017500,47.406250,47.782499,47.751249,1.111002e+08,46.306246
75%,75.206249,72.734375,74.014997,74.381874,1.463086e+08,73.640976
max,145.089996,141.369995,143.600006,143.160004,4.479400e+08,142.704010


In [4]:
# choose a specific stock
# drop feature: volume
# normalize stock data
# create train, validation and test data sets

# function to create train, validation, test data given stock data and sequence length
def load_data(stock, seq_len):
    data_raw = stock
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])

    data = np.array(data)

    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:4]
    y_train = data[:train_set_size,-1,:4]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:4]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:4]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:4]
    y_test = data[train_set_size+valid_set_size:,-1,:4]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

# choose one stock
df_stock = df.copy()
df_stock = df_stock[['Open', 'High', 'Low', 'Close']]

cols = list(df_stock.columns.values)
print('df_stock.columns.values = ', cols)

# min-max normalization of stock
scaler = MinMaxScaler()
df_stock_norm = df_stock.copy()
df_stock_norm = df_stock_norm.values
df_stock_norm = scaler.fit_transform(df_stock_norm)

# create train, test data
seq_len = 60 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df_stock_norm, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)


df_stock.columns.values =  ['Open', 'High', 'Low', 'Close']
x_train.shape =  (982, 29, 4)
y_train.shape =  (982, 4)
x_valid.shape =  (123, 29, 4)
y_valid.shape =  (123, 4)
x_test.shape =  (123, 29, 4)
y_test.shape =  (123, 4)


In [5]:
# 4. Model and validate data 
# RNNs with basic

## Basic Cell RNN in tensorflow

index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = 4 
n_neurons = 200 
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 100 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
tf.compat.v1.reset_default_graph()
# tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# use Basic RNN Cell
layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
          for layer in range(n_layers)]

# use Basic LSTM Cell 
#layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]
                                                                     
multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence
                                              
loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)
                                              
# run graph
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        
        # sess.run(training_op, feed_dict={X: x_batch.reshape(((50 , 19, 4))), y: y_batch.reshape(((, 4)))}) 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 

        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))

    y_train_pred = sess.run(outputs, feed_dict={X: x_train})
    y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
    y_test_pred = sess.run(outputs, feed_dict={X: x_test})
    


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:427: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1692: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


0.00 epochs: MSE train/valid = 0.044596/0.276697
4.99 epochs: MSE train/valid = 0.000170/0.003553
9.98 epochs: MSE train/valid = 0.000121/0.002157
14.97 epochs: MSE train/valid = 0.000111/0.002457
19.96 epochs: MSE train/valid = 0.000099/0.001949
24.95 epochs: MSE train/valid = 0.000085/0.001156
29.94 epochs: MSE train/valid = 0.000082/0.001309
34.93 epochs: MSE train/valid = 0.000070/0.001152
39.92 epochs: MSE train/valid = 0.000066/0.000853
44.91 epochs: MSE train/valid = 0.000064/0.000784
49.90 epochs: MSE train/valid = 0.000059/0.000867
54.89 epochs: MSE train/valid = 0.000064/0.001100
59.88 epochs: MSE train/valid = 0.000070/0.001116
64.87 epochs: MSE train/valid = 0.000083/0.000534
69.86 epochs: MSE train/valid = 0.000052/0.000657
74.85 epochs: MSE train/valid = 0.000082/0.000514
79.84 epochs: MSE train/valid = 0.000070/0.001123
84.83 epochs: MSE train/valid = 0.000050/0.000536
89.82 epochs: MSE train/valid = 0.000054/0.000504
94.81 epochs: MSE train/valid = 0.000052/0.000527
99.

In [6]:
y_test_actual = scaler.inverse_transform(y_test)
y_test_predictions = scaler.inverse_transform(y_test_pred)
save = pd.DataFrame({'Close':y_test_actual[:,3],'Prediction':y_test_predictions[:,3]})
save.index= df.index[-y_test.shape[0]-1:-1]
save.to_csv('../out/RNN/Apple.csv')